In [18]:
import numpy as np
from gym.utils import seeding
from gym import spaces
import gym
from stable_baselines3 import PPO
import pandas as pd
import talib

In [15]:
class TradingEnv(gym.Env):
    def __init__(self, df):
        super(TradingEnv, self).__init__()

        # 株価データ
        self.df = df
        self.reward_range = (0, 1)

        # アクションと観測空間を定義
        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(low=0, high=np.inf, shape=(self.df.shape[1] + 2,))  # holdingsとcashを追加

        # 環境の初期化
        self.reset()

    def step(self, action):
        # 環境の変化と報酬を計算するロジック
        done = False
        obs = self.df.iloc[self.current_step]
        
        previous_cash = self.cash
        previous_holdings = self.holdings

        if action == 0:  # Buy
            self.cash -= obs['close']
            self.holdings += 1
        elif action == 1:  # Hold
            pass
        elif action == 2:  # Sell
            self.cash += obs['close']
            self.holdings -= 1

        self.portfolio_value = self.cash + self.holdings * obs['close']
        reward = self.portfolio_value - (previous_cash + previous_holdings * obs['close'])  # 報酬は資産価値の変化
        
        # 1回の取引で10%以上の損失が出た場合、報酬からペナルティを引く
        if reward < -0.1 * obs['close']:
            reward -= 0.1 * obs['close']
        
        # 長期保有に対するペナルティ（アクションが'Hold'の場合）
        if action == 1:  # Hold
            reward -= 0.01 * obs['close']  # 1%のペナルティ

        self.current_step += 1

        if self.current_step >= len(self.df):
            done = True

        return np.append(obs, [self.holdings, self.cash]), reward, done, {}

    def reset(self):
        # 環境を初期状態にリセットするためのロジックを書く
        self.current_step = 0
        self.holdings = 0  # 保有株式数
        self.cash = 1000000  # 現金
        self.portfolio_value = self.cash
        return np.append(self.df.iloc[self.current_step], [self.holdings, self.cash])


In [16]:
def evaluate(model, num_episodes=100):
    """
    評価関数
    """
    env = model.get_env()  # 環境の取得
    all_rewards = []  # 報酬を保存するリスト

    for i in range(num_episodes):
        obs = env.reset()  # 環境のリセット
        done = False
        total_reward = 0  # 累積報酬の初期化

        while not done:
            action, _ = model.predict(obs)  # 行動の選択
            obs, reward, done, _ = env.step(action)  # 環境のステップ
            total_reward += reward  # 報酬の累積

        all_rewards.append(total_reward)

    mean_reward = np.mean(all_rewards)  # 平均報酬の計算

    print(f"Mean reward: {mean_reward} +/- {np.std(all_rewards)}")

In [19]:
def feature_engineering(df):
    open = df['open'].values
    high = df['high'].values
    low = df['low'].values
    close = df['close'].values
    # volume = df['volume'].values
    hilo = (high + low) / 2

    df['RSI8'] = talib.RSI(close, timeperiod=8) # default = 14
    df['RSI14'] = talib.RSI(close, timeperiod=14)
    df['RSI_ST'] = talib.RSI(close)/close
    df['RSI_LOG'] = log_transform_feature(talib.RSI(close))
    df['MACD'], _, _ = talib.MACD(close)
    df['MACD_ST'], _, _ = talib.MACD(close)/close
    df['ATR'] = talib.ATR(high, low, close)
    
    df['ADX'] = talib.ADX(high, low, close, timeperiod=14)
    df['ADXR'] = talib.ADXR(high, low, close, timeperiod=14)
    df['+DI'] = talib.PLUS_DI(high, low, close, timeperiod=14)
    df['-DI'] = talib.MINUS_DI(high, low, close, timeperiod=14)
    
    df['SMA15'] = talib.SMA(close, timeperiod=15) # 15分足
    df['SMA300'] = talib.SMA(close, timeperiod=300) # 15分足20MA
    
    df['BB_UPPER'], df['BB_MIDDLE'], df['BB_LOWER'] = talib.BBANDS(close, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
    df['BBANDS_upperband'] = (df['BB_UPPER'] - hilo) / close
    df['BBANDS_middleband'] = (df['BB_MIDDLE'] - hilo) / close
    df['BBANDS_lowerband'] = (df['BB_LOWER'] - hilo) / close

    df['STOCH_K'], df['STOCH_D'] = talib.STOCH(high, low, close)/close
    df['MON'] = talib.MOM(close, timeperiod=5)

    df['pinbar'] = pinbar(df)
    df['RCI'] = df['close'].rolling(9).apply(calc_rank_correlation)

    df = df.fillna(method='ffill')
    df = df.dropna()

    return df

def pinbar(df):
    body = np.abs(df['close'] - df['open'])
    upper_wick = df['high'] - np.max(df[['open', 'close']], axis=1)
    lower_wick = np.min(df[['open', 'close']], axis=1) - df['low']
    total_length = df['high'] - df['low']
    
    # 上向きのピンバー（ロングサイン）:下ワックが本体の3倍以上、上ワックが全体の長さの20~30%以内
    is_bullish_pinbar = (lower_wick >= 3 * body) & (upper_wick <= total_length * 0.3)
    
    # 下向きのピンバー（ショートサイン）:上ワックが本体の3倍以上、下ワックが全体の長さの20~30%以内
    is_bearish_pinbar = (upper_wick >= 3 * body) & (lower_wick <= total_length * 0.3)
    
    # 上向きピンバー = 1、下向きピンバー = 2、ピンバーでない = 0
    return np.where(is_bullish_pinbar, 1, np.where(is_bearish_pinbar, 2, 0))

def calc_rank_correlation(series):
    n = len(series)
    ranks = series.rank()
    sum_diffs_squared = sum((ranks - np.arange(n) - 1) ** 2)
    return 1 - 6 * sum_diffs_squared / (n * (n ** 2 - 1))

def log_transform_feature(X):
    X[X <= 0] = np.finfo(float).eps
    return np.log(X)

In [20]:
df = pd.read_csv('../csv/test_combined_data_small.csv')

# 日付は文字列のまま読み込むことができないため、最小の日付からの経過秒数に変換
df['date'] = pd.to_datetime(df['date'])
df['date'] = (df['date'] - df['date'].min()).dt.total_seconds()
display(df)

df = feature_engineering(df)
display(df)

# create environment
env = TradingEnv(df)

# create agent with TensorBoard
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log="./ppo_tensorboard/")

# train agent
model.learn(total_timesteps=10000)

# evaluate model
evaluate(model, num_episodes=100)

model.save("../model/rl_v1.zip")

# モデルの読み込み
# model = PPO.load("../model/rl_v1.zip")

,date,open,high,low,close
0,0.0,4130.3,4132.3,4128.5,4128.9
1,60.0,4128.9,4130.0,4127.0,4128.6
2,120.0,4128.7,4129.3,4128.0,4128.4
3,180.0,4128.4,4129.3,4127.0,4129.2
4,240.0,4129.5,4132.0,4129.5,4130.4
...,...,...,...,...,...
994,59640.0,4154.1,4154.9,4153.7,4154.4
995,59700.0,4154.4,4154.9,4153.5,4153.7
996,59760.0,4153.7,4153.8,4152.5,4152.6
997,59820.0,4152.6,4152.7,4149.1,4149.5


/var/folders/m1/5dpmkc_567gddy0b98vly1w40000gn/T/ipykernel_46871/2621125179.py:63: RuntimeWarning: invalid value encountered in log
  return np.log(X)


,date,open,high,low,close,RSI8,RSI14,RSI_ST,RSI_LOG,MACD,...,BB_MIDDLE,BB_LOWER,BBANDS_upperband,BBANDS_middleband,BBANDS_lowerband,STOCH_K,STOCH_D,MON,pinbar,RCI
0,0.0,4130.3,4132.3,4128.5,4128.9,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
1,60.0,4128.9,4130.0,4127.0,4128.6,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2,120.0,4128.7,4129.3,4128.0,4128.4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
3,180.0,4128.4,4129.3,4127.0,4129.2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4,240.0,4129.5,4132.0,4129.5,4130.4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,59640.0,4154.1,4154.9,4153.7,4154.4,51.976118,57.050393,0.013733,4.043935,1.529690,...,4154.225,4151.113168,0.000731,-0.000018,-0.000767,0.006218,0.006037,-1.2,0,-0.475000
995,59700.0,4154.4,4154.9,4153.5,4153.7,45.734727,53.753870,0.012941,3.984416,1.356626,...,4154.310,4151.357103,0.000737,0.000026,-0.000684,0.003804,0.005481,-0.5,0,-0.558333
996,59760.0,4153.7,4153.8,4152.5,4152.6,37.621399,48.965707,0.011792,3.891120,1.117825,...,4154.350,4151.514863,0.000972,0.000289,-0.000394,0.002618,0.004215,-1.9,0,-0.820833
997,59820.0,4152.6,4152.7,4149.1,4149.5,23.941854,38.545309,0.009289,3.651834,0.670698,...,4154.290,4151.117682,0.001581,0.000817,0.000052,0.001265,0.002564,-5.5,0,-0.820833


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./ppo_tensorboard/PPO_4


ValueError: Expected parameter logits (Tensor of shape (1, 3)) of distribution Categorical(logits: torch.Size([1, 3])) to satisfy the constraint IndependentConstraint(Real(), 1), but found invalid values:
tensor([[nan, nan, nan]])